<a href="https://colab.research.google.com/github/zzzle96/PyTorch/blob/master/convlstm%20basis2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# Install dependencies for video module in pytorch to work
! apt-get install libavformat-dev libavdevice-dev
! pip install av==6.2.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libavdevice-dev is already the newest version (7:3.4.8-0ubuntu0.2).
libavformat-dev is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.


In [15]:
# Download HMDB51 data and splits from serre lab website
! wget http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
! wget http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/test_train_splits.rar

--2020-09-23 19:52:28--  http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
Resolving serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)... 128.148.254.114
Connecting to serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)|128.148.254.114|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar [following]
--2020-09-23 19:52:28--  https://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
Connecting to serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)|128.148.254.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2124008126 (2.0G)
Saving to: ‘hmdb51_org.rar’

hmdb51_org.rar      100%[===================>]   1.98G  40.0MB/s    in 61s     

2020-09-23 19:53:29 (33.4 MB/s) - ‘hmdb51_org.rar’ saved [2124008126/2124008126]

--2020-09-23 19:53:29--  http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/test_train_splits.rar
R

In [16]:
# Import required modules ...

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import random_split, DataLoader
from torch.optim.lr_scheduler import StepLR
import torchvision
from torchvision import get_video_backend
from torchvision.models.video import r3d_18 
from torchvision import transforms
import os
from tqdm.auto import tqdm
import numpy as np
import time
#import av
import random
#print(f"PyAV version -- {av.__version__}")

SEED = 491
torch.manual_seed(SEED)

from collections import OrderedDict
import warnings
warnings.filterwarnings('ignore')

#run_av_diagnostics()

In [17]:
# Extract and organize video data..
! mkdir -p video_data test_train_splits
! unrar e test_train_splits.rar test_train_splits
! rm test_train_splits.rar
! unrar e hmdb51_org.rar 
! rm hmdb51_org.rar
! mv *.rar video_data
for files in os.listdir('video_data'):
    foldername = files.split('.')[0]
    os.system("mkdir -p video_data/" + foldername)
    os.system("unrar e video_data/"+ files + " video_data/"+foldername)

! rm video_data/*.rar 


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from test_train_splits.rar


Would you like to replace the existing file test_train_splits/brush_hair_test_split1.txt
  7194 bytes, modified on 2010-12-13 16:16
with a new one
  7194 bytes, modified on 2010-12-13 16:16

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit a

Extracting  test_train_splits/brush_hair_test_split1.txt                   0%  OK 
Extracting  test_train_splits/brush_hair_test_split2.txt                   1%  OK 
Extracting  test_train_splits/brush_hair_test_split3.txt                   1%  OK 
Extracting  test_train_splits/cartwheel_test_split1.txt                    2%  OK 
Extracting  test_train_splits/cartwheel_test_split2.txt                    2%  OK 
Extracting  test_train_splits/cartwheel_test_split3.txt                    3%  OK 
Extracting  test_train_splits/catch_test_split1.txt                        4%  OK 
Extract

In [18]:
# diagnostics for PyAV installation. For now version 6.2.0 works 
def run_av_diagnostics():
    import av
    av.open("video_data/brush_hair/Aussie_Brunette_Brushing_Hair_II_brush_hair_u_nm_np1_ba_goo_4.avi")
    print(get_video_backend())
    av.logging.set_level(av.logging.ERROR)
    if not hasattr(av.video.frame.VideoFrame, 'pict_type'):
      print("Nah")

run_av_diagnostics()

pyav


In [19]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 9.6 GB  |     Proc size: 3.5 GB
GPU RAM Free: 14162MB | Used: 917MB | Util   6% | Total     15079MB


In [20]:
# download video transformation functions from below link
! wget https://raw.githubusercontent.com/pytorch/vision/6de158c473b83cf43344a0651d7c01128c7850e6/references/video_classification/transforms.py
import transforms as T

--2020-09-23 19:57:40--  https://raw.githubusercontent.com/pytorch/vision/6de158c473b83cf43344a0651d7c01128c7850e6/references/video_classification/transforms.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3102 (3.0K) [text/plain]
Saving to: ‘transforms.py.1’

transforms.py.1     100%[===================>]   3.03K  --.-KB/s    in 0s      

2020-09-23 19:57:40 (54.4 MB/s) - ‘transforms.py.1’ saved [3102/3102]



In [40]:
# Datasets and Dataloaders for model training ..

val_split = 0.05
num_frames = 16 # 16
clip_steps = 1
num_workers = 8
pin_memory = True
train_tfms = torchvision.transforms.Compose([
                                 T.ToFloatTensorInZeroOne(),
                                 T.Resize((128, 171)),
                                 T.RandomHorizontalFlip(),
                                 T.Normalize(mean=[0.43216, 0.394666, 0.37645], std=[0.22803, 0.22145, 0.216989]),
                                 T.RandomCrop((112, 112))
                               ])  
test_tfms =  torchvision.transforms.Compose([
                                             T.ToFloatTensorInZeroOne(),
                                             T.Resize((128, 171)),
                                             T.Normalize(mean=[0.43216, 0.394666, 0.37645], std=[0.22803, 0.22145, 0.216989]),
                                             T.CenterCrop((112, 112))
                                             ])
hmdb51_train = torchvision.datasets.HMDB51('video_data/', 'test_train_splits/', num_frames,
                                                step_between_clips = clip_steps, fold=1, train=True,
                                                transform=train_tfms, num_workers=num_workers)


hmdb51_test = torchvision.datasets.HMDB51('video_data/', 'test_train_splits/', num_frames,
                                                step_between_clips = clip_steps, fold=1, train=False,
                                                transform=test_tfms, num_workers=num_workers)
      
total_train_samples = len(hmdb51_train)
total_val_samples = round(val_split * total_train_samples)

print(f"number of train samples {total_train_samples}")
print(f"number of validation samples {total_val_samples}")
print(f"number of test samples {len(hmdb51_test)}")



number of train samples 291404
number of validation samples 14570
number of test samples 120525


In [41]:
print(hmdb51_train)

Dataset HMDB51
    Number of datapoints: 291404
    Root location: video_data/


In [42]:
import torch
import torch.nn as nn
import torch.nn.functional as F
#from torch.autograd import Variable

class ConvLSTM(nn.Module):
    def __init__(self, n_in, output_channel, class_num, in_ch):
        super(ConvLSTM, self).__init__()
        self.in_ch = in_ch
        self.n_in   = n_in
        self.out_ch = output_channel 
        self.rnn1 = Conv_LSTM(input_channel = self.in_ch, output_channel = self.out_ch)
        self.ap = nn.AvgPool1d(32,32, padding=0)
        self.fcl1 = nn.Linear(self.out_ch * self.n_in // 32, class_num, bias = False)
        
                
    def forward(self, x, hidden):
        h1 = hidden
        x = x.reshape(-1, self.in_ch, self.n_in)
        #print(x)
        x, h1 = self.rnn1(x, h1)
        x = F.relu(x)
        x = self.ap(x)
        x = x.view(-1, self.out_ch * self.n_in // 32)
        x = self.fcl1(x)
        hidden = h1
                
        return x, hidden
        
  

class Conv_LSTM(nn.Module):
    def __init__(self, input_channel, output_channel):
        super(Conv_LSTM, self).__init__()
        self.output_channel = output_channel
        self.fcl_wxf = nn.Conv2d(input_channel,output_channel, 9, stride=1, bias = False, padding = 4)
        self.fcl_wxi = nn.Conv2d(input_channel,output_channel, 9, stride=1, bias = False, padding = 4)
        self.fcl_wxo = nn.Conv2d(input_channel,output_channel, 9, stride=1, bias = False, padding = 4)
        self.fcl_wxg = nn.Conv2d(input_channel,output_channel, 9, stride=1, bias = False, padding = 4)
        self.fcl_whf = nn.Conv2d(output_channel,output_channel, 9, stride=1, bias = False, padding = 4)
        self.fcl_whi = nn.Conv2d(output_channel,output_channel, 9, stride=1, bias = False, padding = 4)
        self.fcl_who = nn.Conv2d(output_channel,output_channel, 9, stride=1, bias = False, padding = 4)
        self.fcl_whg = nn.Conv2d(output_channel,output_channel, 9, stride=1, bias = False, padding = 4)
    
    #def init_hidden(self,batch_size):
    #    # (num_layers * num_directions, batch_size, hidden_size)
    #    hidden = Variable(torch.zeros(1,batch_size,(16,1)))
    #    cell = Variable(torch.zeros(1,batch_size,(16,1)))
    #    return hidden, cell

    def forward(self, input, hidden):
        h_t_1, c_t_1 = self.initHidden(input.size(0),3)
        f_t_i = (self.fcl_wxf(input) + self.fcl_whf(h_t_1))
        i_t_i = (self.fcl_wxi(input) + self.fcl_whi(h_t_1))
        g_t_i = (self.fcl_wxg(input) + self.fcl_whg(h_t_1))
        o_t_i = (self.fcl_wxo(input) + self.fcl_who(h_t_1))

        f_t = torch.sigmoid(f_t_i)
        i_t = torch.sigmoid(i_t_i)
        g_t = torch.tanh(g_t_i)
        o_t = torch.sigmoid(o_t_i)
                
        c_t = f_t * c_t_1 + i_t * g_t
        h_t = o_t * torch.tanh(c_t)
        output = h_t
        hidden = (h_t, c_t)
        
        return output, hidden
        
    def initHidden(self, batch_size, seg_size):
        weight = next(self.parameters())
        return (weight.new_zeros(batch_size, self.output_channel, seg_size),
                weight.new_zeros(batch_size, self.output_channel, seg_size)) 


   
 

In [43]:
bs = 4
lr = 0.001
gamma = 0.7
total_epochs = 10
config = {}
num_workers = 1

kwargs = {'num_workers':num_workers, 'pin_memory':True} if torch.cuda.is_available() else {'num_workers':num_workers}
#kwargs = {'num_workers':num_workers}
#kwargs = {}

hmdb51_train_v1, hmdb51_val_v1 = random_split(hmdb51_train, [total_train_samples - total_val_samples,
                                                                       total_val_samples])

#hmdb51_train_v1.video_clips.compute_clips(16, 1, frame_rate=30)
#hmdb51_val_v1.video_clips.compute_clips(16, 1, frame_rate=30)
#hmdb51_test.video_clips.compute_clips(16, 1, frame_rate=30)

#train_sampler = RandomClipSampler(hmdb51_train_v1.video_clips, 5)
#test_sampler = UniformClipSampler(hmdb51_test.video_clips, 5)
  
train_loader = DataLoader(hmdb51_train_v1, batch_size=bs, shuffle=True, **kwargs)
val_loader   = DataLoader(hmdb51_val_v1, batch_size=bs, shuffle=True, **kwargs)
test_loader  = DataLoader(hmdb51_test, batch_size=bs, shuffle=False, **kwargs)


In [44]:
net = ConvLSTM(16,32,51,3).cuda()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss().to()

In [45]:
epochss = []
costss = []
total_batch = len(train_loader)
epochs = 10
hidden = (0,0)
start = time.time()
for epoch in range(1,total_epochs +1):
    for batch_id, data in enumerate(train_loader):

        data, target = data[0], data[-1]
        print(data.shape)
        

        if torch.cuda.is_available():
           data = data.cuda()
           target = target.cuda()

        optimizer.zero_grad()
        output,hidden = net(data,hidden)
        loss = nn.CrossEntropyLoss()(output, target)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        Loss.update(loss.item(), data.size(0))

        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        num_corrects = pred.eq(target.view_as(pred)).sum().item()
        correct += num_corrects

        Acc.update_acc(num_corrects, data.size(0))

        if flag!= 0 and batch_id%config['log_interval'] == 0:
           print('Train Epoch: {} Batch [{}/{} ({:.0f}%)]\tLoss: {:.6f} Accuracy: {}/{} ({:.0f})%'.format(
                epoch, batch_id * len(data), len(loader.dataset),
                100. * batch_id / len(loader), Loss.avg, correct, Acc.count, 100. * Acc.avg))
        flag = 1

    #total_loss /= len(loader.dataset) 
    print('Train Epoch: {} Average Loss: {:.6f} Average Accuracy: {}/{} ({:.0f})%'.format(
         epoch, Loss.avg, correct, Acc.count, 100. * Acc.avg ))
    print(f"Takes {time.time() - start}")

torch.Size([4, 3, 16, 112, 112])


RuntimeError: ignored